# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
import warnings
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
warnings.filterwarnings('ignore')

# Description of the variables

Variable	                Description
Item_Identifier:            Unique product ID
Item_Weight:                Weight of product
Item_Fat_Content:           Whether the product is low fat or not
Item_Visibility:            The % of total display area of all products in a store allocated to the particular product
Item_Type:	                The category to which the product belongs
Item_MRP:	                Maximum Retail Price (list price) of the product
Outlet_Identifier:          Unique store ID
Outlet_Establishment_Year:	The year in which store was established
Outlet_Size:	            The size of the store in terms of ground area covered
Outlet_Location_Type:	    The type of city in which the store is located
Outlet_Type:	            Whether the outlet is just a grocery store or some sort of supermarket
Item_Outlet_Sales:	        Sales of the product in the particular store. This is the outcome variable to be predicted.

In [2]:
df_train=pd.read_csv("train_v9rqX0R.csv")
df_train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
df_test = pd.read_csv('test_AbJTz2l.csv')
test_data = pd.read_csv('test_AbJTz2l.csv') # backup
df_test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [4]:
#check the number of unique values the following variables contain:
df_train[['Item_Fat_Content', 'Item_Type', 'Outlet_Size','Outlet_Location_Type','Outlet_Type']].nunique()

Item_Fat_Content         5
Item_Type               16
Outlet_Size              3
Outlet_Location_Type     3
Outlet_Type              4
dtype: int64

In [5]:
df_test[['Item_Fat_Content', 'Item_Type', 'Outlet_Size','Outlet_Location_Type','Outlet_Type']].nunique()

Item_Fat_Content         5
Item_Type               16
Outlet_Size              3
Outlet_Location_Type     3
Outlet_Type              4
dtype: int64

In [6]:
df_train['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [7]:
df_test['Item_Fat_Content'].unique()

array(['Low Fat', 'reg', 'Regular', 'LF', 'low fat'], dtype=object)

In [8]:
# Checking the types of item we have in our dataset
df_train['Item_Type'].unique()

array(['Dairy', 'Soft Drinks', 'Meat', 'Fruits and Vegetables',
       'Household', 'Baking Goods', 'Snack Foods', 'Frozen Foods',
       'Breakfast', 'Health and Hygiene', 'Hard Drinks', 'Canned',
       'Breads', 'Starchy Foods', 'Others', 'Seafood'], dtype=object)

In [9]:
df_test['Item_Type'].unique()

array(['Snack Foods', 'Dairy', 'Others', 'Fruits and Vegetables',
       'Baking Goods', 'Health and Hygiene', 'Breads', 'Hard Drinks',
       'Seafood', 'Soft Drinks', 'Household', 'Frozen Foods', 'Meat',
       'Canned', 'Starchy Foods', 'Breakfast'], dtype=object)

In [10]:
# replacing "NaN" of the following columns with suitable measures so that the model designing can be done smoothly
# Item_Weight with "Mean of all other available values"
# Outlet_Type with "Median of all other available values"
df_train['Item_Weight']=df_train["Item_Weight"].replace('nan',np.nan)
df_test['Item_Weight']=df_test["Item_Weight"].replace('nan',np.nan)

In [11]:
df_train["Item_Weight"]=df_train["Item_Weight"].fillna(df_train["Item_Weight"].mean())
df_test["Item_Weight"]=df_test["Item_Weight"].fillna(df_test["Item_Weight"].mean())

In [12]:
df_train["Outlet_Size"].unique()

array(['Medium', nan, 'High', 'Small'], dtype=object)

In [13]:
df_test["Outlet_Size"].unique()

array(['Medium', nan, 'Small', 'High'], dtype=object)

In [14]:
df_train["Outlet_Type"].unique()

array(['Supermarket Type1', 'Supermarket Type2', 'Grocery Store',
       'Supermarket Type3'], dtype=object)

In [15]:
df_test["Outlet_Type"].unique()

array(['Supermarket Type1', 'Grocery Store', 'Supermarket Type3',
       'Supermarket Type2'], dtype=object)

In [16]:
#Replacing the string "nan" with np.nan values
df_train["Outlet_Size"]=df_train['Outlet_Size'].replace("nan", np.nan)
df_test["Outlet_Size"]=df_test['Outlet_Size'].replace("nan", np.nan)

In [17]:
#Replacing the missing values 
df_train["Outlet_Size"]=df_train["Outlet_Size"].fillna(df_train["Outlet_Size"].mode().iat[0])
df_test["Outlet_Size"]=df_test["Outlet_Size"].fillna(df_test["Outlet_Size"].mode().iat[0])

In [18]:
df_train["Outlet_Size"].value_counts()

Medium    5203
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [19]:
df_test["Outlet_Size"].value_counts()

Medium    3468
Small     1592
High       621
Name: Outlet_Size, dtype: int64

In [20]:
df_train["Item_Identifier"].nunique()

1559

In [21]:
df_test["Item_Identifier"].nunique()

1543

In [22]:
#creating dummy variables out of "Item_Identifier" column
df_dum_train_item_id=pd.get_dummies(df_train["Item_Identifier"],prefix="Item_Identifier")
df_dum_test_item_id=pd.get_dummies(df_test["Item_Identifier"], prefix="Item_Identifier")

In [23]:
set(df_train.Item_Identifier.unique())-set(df_test.Item_Identifier.unique())

{'DRE49',
 'FDA04',
 'FDG24',
 'FDG33',
 'FDL10',
 'FDL34',
 'FDO19',
 'FDO52',
 'FDT07',
 'FDU19',
 'FDW13',
 'FDX04',
 'FDX20',
 'NCL31',
 'NCQ06',
 'NCY18'}

In [24]:
df_train=pd.concat([df_train,df_dum_train_item_id], axis=1)
df_test=pd.concat([df_test,df_dum_test_item_id], axis=1)

In [25]:
df_train["Item_Fat_Content"]=df_train["Item_Fat_Content"].replace("Low_Fat", "low fat")
df_train["Item_Fat_Content"]=df_train["Item_Fat_Content"].replace("LF", "low fat")
df_train["Item_Fat_Content"]=df_train["Item_Fat_Content"].replace("reg", "Regular")
df_train["Item_Fat_Content"]=df_train["Item_Fat_Content"].replace("low fat", "Low Fat")

In [26]:
df_test["Item_Fat_Content"]=df_test["Item_Fat_Content"].replace("Low_Fat", "low fat")
df_test["Item_Fat_Content"]=df_test["Item_Fat_Content"].replace("LF", "low fat")
df_test["Item_Fat_Content"]=df_test["Item_Fat_Content"].replace("reg", "Regular")
df_test["Item_Fat_Content"]=df_test["Item_Fat_Content"].replace("low fat", "Low Fat")

In [27]:
df_train["Item_Fat_Content"].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [28]:
df_test["Item_Fat_Content"].value_counts()

Low Fat    3668
Regular    2013
Name: Item_Fat_Content, dtype: int64

In [29]:
from sklearn.preprocessing import LabelEncoder
le_train_item_fat_content = LabelEncoder()
df_train["Item_Fat_Content"] = le_train_item_fat_content.fit_transform(df_train["Item_Fat_Content"])
df_test["Item_Fat_Content"] = le_train_item_fat_content.transform(df_test["Item_Fat_Content"])

In [30]:
df_dum_train_item_type = pd.get_dummies(df_train["Item_Type"],prefix="Item_Type")
df_dum_test_item_type = pd.get_dummies(df_test["Item_Type"],prefix="Item_Type")

In [31]:
# withConcatenating the original dataframe "df_train" and dummy dataframe "df_dum"
df_train=pd.concat([df_train,df_dum_train_item_type], axis=1)
df_test=pd.concat([df_test,df_dum_test_item_type], axis=1)

In [32]:
#Creating dummy variables out of "Outlet_Identifier" column
df_dum_train_out_id = pd.get_dummies(df_train['Outlet_Identifier'], prefix="Outlet_Identifier")
df_dum_test_out_id = pd.get_dummies(df_test['Outlet_Identifier'], prefix="Outlet_Identifier")

In [33]:
#concatenating "df_train" and "df_dum1" to get a consolidated dataframe
df_train=pd.concat([df_train,df_dum_train_out_id], axis=1)
df_test=pd.concat([df_test,df_dum_test_out_id], axis=1)

In [34]:
#Label encoding the column "Outlet_Size"
le_train_out_size = LabelEncoder()
df_train["Outlet_Size"] = le_train_out_size.fit_transform(df_train["Outlet_Size"])
df_test["Outlet_Size"] = le_train_out_size.transform(df_test["Outlet_Size"])

In [35]:
#Checking number of values available in "Outlet_Location_Type"
df_train["Outlet_Location_Type"].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [36]:
# Label encoding the column "Outlet_Location_Type"
le_train_out_loc_type = LabelEncoder()
df_train["Outlet_Location_Type"] = le_train_out_loc_type.fit_transform(df_train["Outlet_Location_Type"])
df_test["Outlet_Location_Type"] = le_train_out_loc_type.transform(df_test["Outlet_Location_Type"])

In [37]:
# Counting the unique values present in "Outlet_Type" column
df_train["Outlet_Type"].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [38]:
df_test["Outlet_Type"].value_counts()

Supermarket Type1    3717
Grocery Store         722
Supermarket Type3     624
Supermarket Type2     618
Name: Outlet_Type, dtype: int64

In [39]:
#label Encoding the column "Outlet_Type"
le_train_out_type = LabelEncoder()
df_train["Outlet_Type"] = le_train_out_type.fit_transform(df_train["Outlet_Type"])
df_test["Outlet_Type"] = le_train_out_type.transform(df_test["Outlet_Type"])

In [40]:
#dropping the rendundant columns as these columns have already been converted using either label encoding or onehot encoding
df_train.drop(["Item_Type","Outlet_Identifier", "Item_Identifier"], axis=1, inplace=True)
df_test.drop(["Item_Type","Outlet_Identifier", "Item_Identifier"], axis=1, inplace=True)

In [41]:
#Checking the datatype of each columns 
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Columns: 1594 entries, Item_Weight to Outlet_Identifier_OUT049
dtypes: float64(4), int32(4), int64(1), uint8(1585)
memory usage: 13.3 MB


In [42]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Columns: 1577 entries, Item_Weight to Outlet_Identifier_OUT049
dtypes: float64(3), int32(4), int64(1), uint8(1569)
memory usage: 8.8 MB


In [43]:
df_test.dtypes.unique()

array([dtype('float64'), dtype('int32'), dtype('int64'), dtype('uint8')],
      dtype=object)

In [44]:
#Binning the column "Outlet_Establishment_Year" into interval of 10 years
df_train['Outlet_Establishment_Year']=pd.cut(df_train.Outlet_Establishment_Year, [1980,1990,2000,2010,2020], labels=['1980','1990','2000','2010'])
df_test['Outlet_Establishment_Year']=pd.cut(df_test.Outlet_Establishment_Year, [1980,1990,2000,2010,2020], labels=['1980','1990','2000','2010'])

In [45]:
#label encoding the updated "Outlet_Establishment_Year"
le_train_out_est_year = LabelEncoder()
df_train["Outlet_Establishment_Year"] = le_train_out_est_year.fit_transform(df_train["Outlet_Establishment_Year"])
df_test["Outlet_Establishment_Year"] = le_train_out_est_year.transform(df_test["Outlet_Establishment_Year"])

In [46]:
df_train.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Identifier_DRA12,...,Outlet_Identifier_OUT010,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049
0,9.30,0,0.016047,249.8092,1,1,0,1,3735.1380,0,...,0,0,0,0,0,0,0,0,0,1
1,5.92,1,0.019278,48.2692,2,1,2,2,443.4228,0,...,0,0,0,1,0,0,0,0,0,0
2,17.50,0,0.016760,141.6180,1,1,0,1,2097.2700,0,...,0,0,0,0,0,0,0,0,0,1
3,19.20,1,0.000000,182.0950,1,1,2,0,732.3800,0,...,1,0,0,0,0,0,0,0,0,0
4,8.93,0,0.000000,53.8614,0,0,2,1,994.7052,0,...,0,1,0,0,0,0,0,0,0,0


# Modeling

## Train test dataset preparation

In [47]:
# Setting up X and y for modelling part
X = df_train.drop('Item_Outlet_Sales', axis=1)
y = df_train["Item_Outlet_Sales"]

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred)/y_true))

def symmetric_mean_absolute_percentage_error(y_true,y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred)/((y_true + y_pred)/200)))

def performance_measure(model,scaler=None,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test):
    if scaler:
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
    model.fit(X_train,y_train)
    
    y_pred_train = model.predict(X_train)
    y_pred_train[y_pred_train<0] = 0
    y_pred_train[y_pred_train>1.5e4] = 1.5e4
    
    y_pred_test = model.predict(X_test)
    y_pred_test[y_pred_test<0] = 0
    y_pred_test[y_pred_test>1.5e4] = 1.5e4
    
    mae_train = mean_absolute_error(y_train,y_pred_train)
    rmse_train = mean_squared_error(y_train,y_pred_train, squared=False)
    r2_train=r2_score(y_train, y_pred_train)
    mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
    smape_train = symmetric_mean_absolute_percentage_error(y_train, y_pred_train)
    print("*****Train metrics*****")
    print("MAE", mae_train)
    print("RMSE", rmse_train)
    print("r2_score", r2_train)
    print("MAPE", mape_train)
    print("SMAPE", smape_train)
    
    mae_test = mean_absolute_error(y_test,y_pred_test)
    rmse_test = mean_squared_error(y_test,y_pred_test, squared=False)
    r2_test = r2_score(y_test, y_pred_test)
    mape_test = mean_absolute_percentage_error(y_test, y_pred_test)
    smape_test = symmetric_mean_absolute_percentage_error(y_test, y_pred_test)
    print("*****Test metrics*****")
    print("MAE", mae_test)
    print("RMSE", rmse_test)
    print("r2_score", r2_test)
    print("MAPE", mape_test)
    print("SMAPE", smape_test)

In [50]:
def solution(model,output_filename,df_test=[],test_data=[]):
    model.fit(X_train,y_train)
    pred = model.predict(df_test)
    output = pd.DataFrame([test_data.Item_Identifier,test_data.Outlet_Identifier,pred]).transpose()
    output.columns = ['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']
    output.to_csv(output_filename,index=None)
    return output

In [95]:
def solution_combined(model1, model2,output_filename,df_test=[],test_data=[]):
    model1.fit(X_train,y_train)
    model2.fit(X_train,y_train)
    pred1 = model1.predict(df_test)
    pred2 = model2.predict(df_test)
    pred  = (pred1 + pred2)/2
    output = pd.DataFrame([test_data.Item_Identifier,test_data.Outlet_Identifier,pred]).transpose()
    output.columns = ['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']
    output.to_csv(output_filename,index=None)
    return output

In [51]:
reg = LinearRegression()

In [52]:
performance_measure(reg,StandardScaler())

*****Train metrics*****
MAE 743.6451271346992
RMSE 1011.3287244781137
r2_score 0.6542365046601075
MAPE 0.651173503569604
SMAPE 48.40275730543334
*****Test metrics*****
MAE 951.2980636104403
RMSE 1404.0109522026814
r2_score 0.2747356646356113
MAPE 0.8624387329970441
SMAPE 59.744108582661795


In [53]:
performance_measure(reg,MinMaxScaler())

*****Train metrics*****
MAE 713.0572540801921
RMSE 980.0568572396451
r2_score 0.6752890022736941
MAPE 0.607124010455629
SMAPE 46.47872011056186
*****Test metrics*****
MAE 923.0515716239003
RMSE 1392.4936024538354
r2_score 0.28658580287878643
MAPE 0.8297591672311648
SMAPE 57.31500166026164


In [54]:
performance_measure(reg)

*****Train metrics*****
MAE 713.0593081642659
RMSE 980.0615883338216
r2_score 0.675285867267787
MAPE 0.6071399137500227
SMAPE 46.47803492467785
*****Test metrics*****
MAE 940.5704641307161
RMSE 1483.8256307024906
r2_score 0.1899327629202605
MAPE 0.8432509778194522
SMAPE 57.30477375046702


## Decision Tree 

In [55]:
from sklearn.tree import DecisionTreeRegressor
reg_dt=DecisionTreeRegressor(min_samples_split=2,max_depth=5)

In [56]:
performance_measure(reg_dt)#,StandardScaler())

*****Train metrics*****
MAE 762.5535996251765
RMSE 1086.5470084045685
r2_score 0.6008910237066474
MAPE 0.5480398308611981
SMAPE 37.95872667824018
*****Test metrics*****
MAE 722.6197992533215
RMSE 1028.92624452797
r2_score 0.610485328855638
MAPE 0.5591183981628167
SMAPE 38.67118607143934


In [57]:
null_columns = list(set(df_train.columns)-set(df_test.columns))
for col in null_columns:
    df_test[col]=0
df_test = df_test[X_train.columns]

In [58]:
solution(reg_dt,'output_dt_minsamp2_max_dep5.csv',df_test,test_data)

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1507.08
1,FDW14,OUT017,1507.08
2,NCN55,OUT010,532.228
3,FDQ58,OUT017,2499.48
4,FDY38,OUT027,6458.3
...,...,...,...
5676,FDB58,OUT046,2026.85
5677,FDD47,OUT018,3001.89
5678,NCO17,OUT045,2026.85
5679,FDJ26,OUT017,3570.73


In [59]:
for min_samples_split,max_depth,min_samples_leaf in product(range(2,6),range(3,7),range(1,5)):
    print(min_samples_split,max_depth,min_samples_leaf)
    reg_dt = DecisionTreeRegressor(min_samples_split=min_samples_split,max_depth=max_depth,min_samples_leaf=min_samples_leaf)
    performance_measure(reg_dt)

2 3 1
*****Train metrics*****
MAE 840.3986948858201
RMSE 1189.2051715605935
r2_score 0.5219117907786619
MAPE 0.6144660906959523
SMAPE 41.49750834446553
*****Test metrics*****
MAE 795.142590307858
RMSE 1122.4735940795977
r2_score 0.5364382467337739
MAPE 0.6299791488921955
SMAPE 41.93843612027422
2 3 2
*****Train metrics*****
MAE 840.39869488582
RMSE 1189.2051715605935
r2_score 0.5219117907786619
MAPE 0.6144660906959519
SMAPE 41.49750834446553
*****Test metrics*****
MAE 795.142590307858
RMSE 1122.4735940795977
r2_score 0.5364382467337738
MAPE 0.6299791488921951
SMAPE 41.93843612027422
2 3 3
*****Train metrics*****
MAE 840.39869488582
RMSE 1189.2051715605937
r2_score 0.5219117907786617
MAPE 0.614466090695952
SMAPE 41.49750834446553
*****Test metrics*****
MAE 795.142590307858
RMSE 1122.4735940795977
r2_score 0.5364382467337739
MAPE 0.6299791488921952
SMAPE 41.93843612027422
2 3 4
*****Train metrics*****
MAE 840.3986948858201
RMSE 1189.2051715605937
r2_score 0.5219117907786617
MAPE 0.614466

*****Train metrics*****
MAE 750.6556731718808
RMSE 1071.8435488699731
r2_score 0.6116196475256821
MAPE 0.5345435549908878
SMAPE 37.32114019980665
*****Test metrics*****
MAE 724.3936771198564
RMSE 1042.121941153975
r2_score 0.60043042652549
MAPE 0.5593955685068863
SMAPE 38.55717003973001
3 6 2
*****Train metrics*****
MAE 750.3175451946017
RMSE 1071.7487601070197
r2_score 0.6116883375231603
MAPE 0.5340032171598471
SMAPE 37.20850439469311
*****Test metrics*****
MAE 724.6029136604376
RMSE 1044.8475542250574
r2_score 0.5983375884161154
MAPE 0.5609363586367048
SMAPE 38.637207941861334
3 6 3
*****Train metrics*****
MAE 751.023071656527
RMSE 1072.6209391398784
r2_score 0.6110560716246016
MAPE 0.5325854470799054
SMAPE 37.12473871521554
*****Test metrics*****
MAE 726.7328616766245
RMSE 1046.5264818879407
r2_score 0.5970457177786153
MAPE 0.5651887412663767
SMAPE 38.83479170904889
3 6 4
*****Train metrics*****
MAE 751.8706613677326
RMSE 1073.703043329842
r2_score 0.6102709104578758
MAPE 0.53310192

*****Train metrics*****
MAE 762.5535996251765
RMSE 1086.5470084045685
r2_score 0.6008910237066474
MAPE 0.548039830861198
SMAPE 37.95872667824018
*****Test metrics*****
MAE 722.6197992533213
RMSE 1028.92624452797
r2_score 0.610485328855638
MAPE 0.5591183981628164
SMAPE 38.67118607143934
5 5 2
*****Train metrics*****
MAE 763.0839070010879
RMSE 1086.803645097433
r2_score 0.6007024665559333
MAPE 0.5475932958877505
SMAPE 37.935519726910925
*****Test metrics*****
MAE 721.1493027181338
RMSE 1027.484851615141
r2_score 0.6115758840520511
MAPE 0.5586892538942666
SMAPE 38.5662800130845
5 5 3
*****Train metrics*****
MAE 763.0908066943074
RMSE 1086.8107108518773
r2_score 0.6006972745463084
MAPE 0.5471240641065851
SMAPE 37.90669691873477
*****Test metrics*****
MAE 721.6636689847847
RMSE 1027.7334888068726
r2_score 0.6113878747274326
MAPE 0.5598153409266379
SMAPE 38.60736948162274
5 5 4
*****Train metrics*****
MAE 763.0994137002455
RMSE 1086.810855119369
r2_score 0.6006971685362996
MAPE 0.54714144058

In [60]:
reg_dt = DecisionTreeRegressor(max_depth=5,min_samples_split=4)
solution(reg_dt,'output_dt_minsamp4_max_dep6.csv',df_test,test_data)

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1507.08
1,FDW14,OUT017,1507.08
2,NCN55,OUT010,532.228
3,FDQ58,OUT017,2499.48
4,FDY38,OUT027,6458.3
...,...,...,...
5676,FDB58,OUT046,2026.85
5677,FDD47,OUT018,3001.89
5678,NCO17,OUT045,2026.85
5679,FDJ26,OUT017,3570.73


## Random Forest Regressor

In [106]:
from sklearn.ensemble import RandomForestRegressor
reg_rf = RandomForestRegressor(n_estimators = 120,min_samples_split=2,min_samples_leaf=10,random_state=0)

In [107]:
performance_measure(reg_rf, scaler=StandardScaler())

*****Train metrics*****
MAE 647.7500039309506
RMSE 932.1053950319867
r2_score 0.7062861012620854
MAPE 0.4641750091762925
SMAPE 33.15249445584221
*****Test metrics*****
MAE 724.5207090852585
RMSE 1039.2116493055487
r2_score 0.6026590339176643
MAPE 0.5611622140315762
SMAPE 38.55031408388929


In [63]:
for n_estimators,min_samples_split,min_samples_leaf in product([60,80,100,120,150],range(3,7),range(1,7)):
    print(n_estimators,min_samples_split,min_samples_leaf)
    reg_rf = RandomForestRegressor(n_estimators=n_estimators,min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf,random_state=0)
    performance_measure(reg_rf)

60 3 1
*****Train metrics*****
MAE 319.14125577912125
RMSE 467.88220307255676
r2_score 0.9259939018967704
MAPE 0.21882447882856845
SMAPE 17.7788919087332
*****Test metrics*****
MAE 766.454488064939
RMSE 1101.829828233907
r2_score 0.553332472164728
MAPE 0.5600021120829116
SMAPE 39.933755915775166
60 3 2
*****Train metrics*****
MAE 392.78744742018336
RMSE 583.6640312432578
r2_score 0.8848350520781815
MAPE 0.27831148326226296
SMAPE 21.544969888049298
*****Test metrics*****
MAE 759.0599047788236
RMSE 1086.3262176180147
r2_score 0.5658139633034172
MAPE 0.5718098269341996
SMAPE 39.64592963160367
60 3 3
*****Train metrics*****
MAE 468.4038642138587
RMSE 688.0950771457018
r2_score 0.8399368511574075
MAPE 0.3339885753774919
SMAPE 25.134024236634247
*****Test metrics*****
MAE 743.5310992891733
RMSE 1064.4289560927357
r2_score 0.5831414714662331
MAPE 0.5649303951633978
SMAPE 39.14289340335603
60 3 4
*****Train metrics*****
MAE 519.774652869302
RMSE 758.8515380707212
r2_score 0.805325937388012
MAP

*****Train metrics*****
MAE 556.8782226047647
RMSE 809.1110492131929
r2_score 0.7786850669354877
MAPE 0.39882671425806926
SMAPE 29.173835813770395
*****Test metrics*****
MAE 731.9986403501059
RMSE 1048.3030691049769
r2_score 0.5956764431689079
MAPE 0.5630762743898572
SMAPE 38.753780523098015
80 3 6
*****Train metrics*****
MAE 584.9344948633102
RMSE 846.5065134459933
r2_score 0.7577548654817399
MAPE 0.4194237893140457
SMAPE 30.42960353910008
*****Test metrics*****
MAE 731.1912188241603
RMSE 1047.2675113933446
r2_score 0.5964748641307713
MAPE 0.5639464030589266
SMAPE 38.77861322610255
80 4 1
*****Train metrics*****
MAE 338.89518572172904
RMSE 495.6887188897025
r2_score 0.9169360604316927
MAPE 0.2320782337305334
SMAPE 18.68268075930083
*****Test metrics*****
MAE 765.5937795512754
RMSE 1098.5040311468663
r2_score 0.5560248726550523
MAPE 0.5590705448006899
SMAPE 39.81847128884286
80 4 2
*****Train metrics*****
MAE 390.843985407901
RMSE 580.9323342586182
r2_score 0.885910532251357
MAPE 0.277

KeyboardInterrupt: 

In [64]:
#reg_rf = RandomForestRegressor(n_estimators=60,min_samples_split=3,min_samples_leaf=6,random_state=0)

In [108]:
solution(reg_rf,'output_rf_nest604_minsampspl3_minsampleaf6.csv',df_test,test_data)

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1662.4
1,FDW14,OUT017,1247.2
2,NCN55,OUT010,545.74
3,FDQ58,OUT017,2225.33
4,FDY38,OUT027,6667.47
...,...,...,...
5676,FDB58,OUT046,2067.26
5677,FDD47,OUT018,2811.96
5678,NCO17,OUT045,1757.56
5679,FDJ26,OUT017,4025.83


# Gradiant Boosting

In [66]:
from sklearn.ensemble import GradientBoostingRegressor
reg_gb=GradientBoostingRegressor(n_estimators=70, verbose=3)

In [89]:
performance_measure(reg_gb, StandardScaler())

      Iter       Train Loss   Remaining Time 
         1     2664720.3897           11.75s
         2     2421067.0994           12.75s
         3     2220744.1422           12.37s
         4     2059000.0015           12.45s
         5     1919084.3311           12.52s
         6     1802553.6473           12.26s
         7     1707113.5037           11.99s
         8     1628374.5666           11.68s
         9     1560438.3543           11.63s
        10     1501005.4229           11.40s
        11     1450671.5756           11.19s
        12     1409197.3837           11.10s
        13     1374077.2806           10.91s
        14     1344325.9995           10.81s
        15     1318753.2648           10.52s
        16     1297628.5755           10.38s
        17     1279309.8123           10.17s
        18     1262720.9687           10.05s
        19     1249069.2408            9.83s
        20     1237690.2182            9.64s
        21     1226970.6468            9.49s
        2

In [90]:
solution(reg_gb,'output_gb.csv',df_test,test_data)

      Iter       Train Loss   Remaining Time 
         1     2664720.3897           12.51s
         2     2421067.0994           12.97s
         3     2220744.1422           12.49s
         4     2059000.0015           12.35s
         5     1919084.3311           12.39s
         6     1802553.6473           12.42s
         7     1707113.5037           12.16s
         8     1628374.5666           11.86s
         9     1560438.3543           11.65s
        10     1501005.4229           11.49s
        11     1450671.5756           11.23s
        12     1409197.3837           11.12s
        13     1374077.2806           10.91s
        14     1344325.9995           10.75s
        15     1318753.2648           10.54s
        16     1297628.5755           10.36s
        17     1279309.8123           10.26s
        18     1262720.9687           10.06s
        19     1249069.2408            9.94s
        20     1237690.2182            9.77s
        21     1226970.6468            9.61s
        2

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1627.96
1,FDW14,OUT017,1366.08
2,NCN55,OUT010,705.604
3,FDQ58,OUT017,2460.3
4,FDY38,OUT027,5995.01
...,...,...,...
5676,FDB58,OUT046,2150.35
5677,FDD47,OUT018,2540.1
5678,NCO17,OUT045,1909.14
5679,FDJ26,OUT017,3422.18


## combining Random Forest and Gradiant boosting

In [96]:
solution_combined(reg_rf,reg_gb, "output_gb_n_rf.csv",df_test,test_data)

      Iter       Train Loss   Remaining Time 
         1     2664720.3897           10.12s
         2     2421067.0994           10.11s
         3     2220744.1422           10.05s
         4     2059000.0015            9.96s
         5     1919084.3311            9.79s
         6     1802553.6473            9.67s
         7     1707113.5037            9.50s
         8     1628374.5666            9.33s
         9     1560438.3543            9.16s
        10     1501005.4229            9.02s
        11     1450671.5756            8.89s
        12     1409197.3837            8.78s
        13     1374077.2806            8.63s
        14     1344325.9995            8.51s
        15     1318753.2648            8.34s
        16     1297628.5755            8.18s
        17     1279309.8123            8.05s
        18     1262720.9687            7.90s
        19     1249069.2408            7.77s
        20     1237690.2182            7.61s
        21     1226970.6468            7.47s
        2

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1664.31
1,FDW14,OUT017,1304.6
2,NCN55,OUT010,601.335
3,FDQ58,OUT017,2339.16
4,FDY38,OUT027,6199.86
...,...,...,...
5676,FDB58,OUT046,2118.24
5677,FDD47,OUT018,2681.35
5678,NCO17,OUT045,1792.99
5679,FDJ26,OUT017,3814.96


## XG Boosting

In [69]:
import xgboost as xg
reg_xgb=xg.XGBRegressor(objective='reg:linear')

In [71]:
performance_measure(reg_xgb, scaler=StandardScaler())

[10:46:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
*****Train metrics*****
MAE 652.5080401001281
RMSE 905.5804143726348
r2_score 0.7227647194954406
MAPE 0.49404054473446396
SMAPE 34.70391437698945
*****Test metrics*****
MAE 750.3946106035429
RMSE 1087.298707160696
r2_score 0.5650362404230138
MAPE 0.5968098044558205
SMAPE 40.199088888859784


In [72]:
solution(reg_xgb,'output_xgb.csv',df_test,test_data)

[10:48:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.


,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1538.49
1,FDW14,OUT017,1374.49
2,NCN55,OUT010,639.059
3,FDQ58,OUT017,2443.25
4,FDY38,OUT027,6623.58
...,...,...,...
5676,FDB58,OUT046,2132.25
5677,FDD47,OUT018,2718.46
5678,NCO17,OUT045,1928.48
5679,FDJ26,OUT017,2939.73


In [86]:
import lightgbm as lgb
reg_lgb= lgb.LGBMRegressor(num_leaves=15)

In [87]:
performance_measure(reg_lgb, scaler=StandardScaler())

*****Train metrics*****
MAE 705.4310119560887
RMSE 993.5777291643603
r2_score 0.6662677692516423
MAPE 0.5333624955530678
SMAPE 36.77032154323994
*****Test metrics*****
MAE 717.5127669513598
RMSE 1033.2973634449484
r2_score 0.607168800563231
MAPE 0.5699217554666116
SMAPE 38.431990874438064


In [88]:
solution(reg_lgb,'output_lgb.csv',df_test,test_data)

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1546.74
1,FDW14,OUT017,1311.39
2,NCN55,OUT010,530.433
3,FDQ58,OUT017,2272.65
4,FDY38,OUT027,6496.86
...,...,...,...
5676,FDB58,OUT046,2122.98
5677,FDD47,OUT018,2464.97
5678,NCO17,OUT045,1835.03
5679,FDJ26,OUT017,4069.96


## Catboost Regression

In [92]:
import catboost as cb
reg_cb=cb.CatBoostRegressor()

In [93]:
performance_measure(reg_cb, scaler=StandardScaler())

Learning rate set to 0.054928
0:	learn: 1667.4351902	total: 184ms	remaining: 3m 3s
1:	learn: 1619.3854797	total: 197ms	remaining: 1m 38s
2:	learn: 1574.1285607	total: 207ms	remaining: 1m 8s
3:	learn: 1533.8483176	total: 220ms	remaining: 54.7s
4:	learn: 1495.4441838	total: 231ms	remaining: 46s
5:	learn: 1462.4540060	total: 240ms	remaining: 39.8s
6:	learn: 1429.5135555	total: 250ms	remaining: 35.4s
7:	learn: 1400.8985540	total: 259ms	remaining: 32.1s
8:	learn: 1374.8322394	total: 267ms	remaining: 29.4s
9:	learn: 1350.3749634	total: 279ms	remaining: 27.6s
10:	learn: 1326.9590485	total: 289ms	remaining: 26s
11:	learn: 1304.9557712	total: 299ms	remaining: 24.6s
12:	learn: 1285.9208633	total: 308ms	remaining: 23.4s
13:	learn: 1268.7546225	total: 316ms	remaining: 22.3s
14:	learn: 1252.8749351	total: 324ms	remaining: 21.3s
15:	learn: 1238.2544486	total: 333ms	remaining: 20.5s
16:	learn: 1224.1407072	total: 342ms	remaining: 19.8s
17:	learn: 1211.3477044	total: 351ms	remaining: 19.2s
18:	learn: 

163:	learn: 1056.8116632	total: 1.84s	remaining: 9.4s
164:	learn: 1056.6020819	total: 1.86s	remaining: 9.39s
165:	learn: 1056.3992533	total: 1.87s	remaining: 9.39s
166:	learn: 1056.2509563	total: 1.88s	remaining: 9.39s
167:	learn: 1056.1405723	total: 1.89s	remaining: 9.38s
168:	learn: 1055.9856908	total: 1.91s	remaining: 9.37s
169:	learn: 1055.7871311	total: 1.92s	remaining: 9.36s
170:	learn: 1055.5955375	total: 1.93s	remaining: 9.35s
171:	learn: 1055.4592829	total: 1.94s	remaining: 9.34s
172:	learn: 1055.3206049	total: 1.95s	remaining: 9.33s
173:	learn: 1055.0930873	total: 1.98s	remaining: 9.38s
174:	learn: 1054.8666487	total: 1.99s	remaining: 9.37s
175:	learn: 1054.7399761	total: 2s	remaining: 9.36s
176:	learn: 1054.5980222	total: 2.01s	remaining: 9.35s
177:	learn: 1054.2977353	total: 2.02s	remaining: 9.34s
178:	learn: 1054.1312682	total: 2.03s	remaining: 9.33s
179:	learn: 1053.9391404	total: 2.04s	remaining: 9.32s
180:	learn: 1053.7394323	total: 2.06s	remaining: 9.31s
181:	learn: 10

319:	learn: 1031.7014304	total: 3.62s	remaining: 7.7s
320:	learn: 1031.5632456	total: 3.63s	remaining: 7.69s
321:	learn: 1031.3073368	total: 3.64s	remaining: 7.67s
322:	learn: 1031.1735941	total: 3.65s	remaining: 7.66s
323:	learn: 1030.9602659	total: 3.66s	remaining: 7.64s
324:	learn: 1030.8550771	total: 3.67s	remaining: 7.63s
325:	learn: 1030.6120944	total: 3.68s	remaining: 7.62s
326:	learn: 1030.4783176	total: 3.69s	remaining: 7.6s
327:	learn: 1030.3839404	total: 3.7s	remaining: 7.59s
328:	learn: 1030.2577588	total: 3.72s	remaining: 7.58s
329:	learn: 1030.0771555	total: 3.73s	remaining: 7.56s
330:	learn: 1029.9494752	total: 3.73s	remaining: 7.55s
331:	learn: 1029.7963460	total: 3.74s	remaining: 7.53s
332:	learn: 1029.5941424	total: 3.75s	remaining: 7.52s
333:	learn: 1029.3690946	total: 3.76s	remaining: 7.5s
334:	learn: 1029.1904291	total: 3.78s	remaining: 7.51s
335:	learn: 1028.9797097	total: 3.79s	remaining: 7.5s
336:	learn: 1028.7636698	total: 3.81s	remaining: 7.49s
337:	learn: 102

473:	learn: 1009.2846240	total: 5.25s	remaining: 5.83s
474:	learn: 1009.1516745	total: 5.26s	remaining: 5.81s
475:	learn: 1009.0154096	total: 5.27s	remaining: 5.8s
476:	learn: 1008.8453611	total: 5.28s	remaining: 5.79s
477:	learn: 1008.6948579	total: 5.29s	remaining: 5.78s
478:	learn: 1008.5669607	total: 5.3s	remaining: 5.76s
479:	learn: 1008.5187353	total: 5.31s	remaining: 5.75s
480:	learn: 1008.3993052	total: 5.32s	remaining: 5.74s
481:	learn: 1008.2323652	total: 5.33s	remaining: 5.73s
482:	learn: 1008.1194412	total: 5.34s	remaining: 5.72s
483:	learn: 1008.0017681	total: 5.35s	remaining: 5.7s
484:	learn: 1007.8197491	total: 5.36s	remaining: 5.69s
485:	learn: 1007.6990184	total: 5.37s	remaining: 5.68s
486:	learn: 1007.6057898	total: 5.38s	remaining: 5.67s
487:	learn: 1007.4484767	total: 5.39s	remaining: 5.66s
488:	learn: 1007.2758984	total: 5.4s	remaining: 5.64s
489:	learn: 1007.1902368	total: 5.41s	remaining: 5.63s
490:	learn: 1006.9687967	total: 5.42s	remaining: 5.62s
491:	learn: 10

630:	learn: 991.0451981	total: 6.87s	remaining: 4.01s
631:	learn: 990.9692121	total: 6.88s	remaining: 4s
632:	learn: 990.8800100	total: 6.89s	remaining: 3.99s
633:	learn: 990.7497785	total: 6.9s	remaining: 3.98s
634:	learn: 990.6339690	total: 6.91s	remaining: 3.97s
635:	learn: 990.5699120	total: 6.92s	remaining: 3.96s
636:	learn: 990.4258110	total: 6.93s	remaining: 3.95s
637:	learn: 990.3108230	total: 6.94s	remaining: 3.94s
638:	learn: 990.1894729	total: 6.96s	remaining: 3.93s
639:	learn: 990.0914363	total: 6.97s	remaining: 3.92s
640:	learn: 989.9461373	total: 6.98s	remaining: 3.91s
641:	learn: 989.7939055	total: 6.99s	remaining: 3.9s
642:	learn: 989.6719022	total: 7s	remaining: 3.89s
643:	learn: 989.4719157	total: 7.02s	remaining: 3.88s
644:	learn: 989.3728930	total: 7.03s	remaining: 3.87s
645:	learn: 989.1926348	total: 7.04s	remaining: 3.86s
646:	learn: 989.0065273	total: 7.05s	remaining: 3.85s
647:	learn: 988.9051010	total: 7.07s	remaining: 3.84s
648:	learn: 988.8124196	total: 7.08s

794:	learn: 973.6684255	total: 8.46s	remaining: 2.18s
795:	learn: 973.5722842	total: 8.47s	remaining: 2.17s
796:	learn: 973.4405143	total: 8.48s	remaining: 2.16s
797:	learn: 973.3204597	total: 8.49s	remaining: 2.15s
798:	learn: 973.1170631	total: 8.5s	remaining: 2.14s
799:	learn: 973.0139955	total: 8.51s	remaining: 2.13s
800:	learn: 972.8938502	total: 8.52s	remaining: 2.12s
801:	learn: 972.7526607	total: 8.53s	remaining: 2.1s
802:	learn: 972.6164429	total: 8.54s	remaining: 2.09s
803:	learn: 972.5157173	total: 8.55s	remaining: 2.08s
804:	learn: 972.4190732	total: 8.55s	remaining: 2.07s
805:	learn: 972.3484446	total: 8.56s	remaining: 2.06s
806:	learn: 972.2259260	total: 8.57s	remaining: 2.05s
807:	learn: 972.1298369	total: 8.58s	remaining: 2.04s
808:	learn: 972.0958947	total: 8.59s	remaining: 2.03s
809:	learn: 972.0048588	total: 8.6s	remaining: 2.02s
810:	learn: 971.9170739	total: 8.61s	remaining: 2s
811:	learn: 971.7626129	total: 8.62s	remaining: 1.99s
812:	learn: 971.6865163	total: 8.6

958:	learn: 957.5410980	total: 9.9s	remaining: 423ms
959:	learn: 957.4565278	total: 9.91s	remaining: 413ms
960:	learn: 957.4342720	total: 9.92s	remaining: 403ms
961:	learn: 957.2974673	total: 9.93s	remaining: 392ms
962:	learn: 957.2146287	total: 9.94s	remaining: 382ms
963:	learn: 957.0942539	total: 9.94s	remaining: 371ms
964:	learn: 956.9725594	total: 9.95s	remaining: 361ms
965:	learn: 956.8820382	total: 9.96s	remaining: 351ms
966:	learn: 956.7777698	total: 9.97s	remaining: 340ms
967:	learn: 956.7205392	total: 9.98s	remaining: 330ms
968:	learn: 956.6898712	total: 9.99s	remaining: 319ms
969:	learn: 956.3103270	total: 9.99s	remaining: 309ms
970:	learn: 956.1875910	total: 10s	remaining: 299ms
971:	learn: 956.1015099	total: 10s	remaining: 288ms
972:	learn: 956.0293479	total: 10s	remaining: 278ms
973:	learn: 955.9048819	total: 10s	remaining: 268ms
974:	learn: 955.7605341	total: 10s	remaining: 257ms
975:	learn: 955.6927419	total: 10s	remaining: 247ms
976:	learn: 955.6195999	total: 10.1s	rema

In [94]:
solution(reg_cb,'output_cb.csv',df_test,test_data)

Learning rate set to 0.054928
0:	learn: 1667.4351902	total: 9.46ms	remaining: 9.45s
1:	learn: 1619.3854797	total: 18.8ms	remaining: 9.4s
2:	learn: 1574.1285607	total: 28ms	remaining: 9.3s
3:	learn: 1533.8483176	total: 37.1ms	remaining: 9.23s
4:	learn: 1495.4441838	total: 46ms	remaining: 9.15s
5:	learn: 1462.4540060	total: 54.8ms	remaining: 9.08s
6:	learn: 1429.5135555	total: 63.8ms	remaining: 9.05s
7:	learn: 1400.8985540	total: 73ms	remaining: 9.05s
8:	learn: 1374.8322394	total: 82.1ms	remaining: 9.04s
9:	learn: 1350.3749634	total: 91.1ms	remaining: 9.02s
10:	learn: 1326.9590485	total: 99.7ms	remaining: 8.96s
11:	learn: 1304.9557712	total: 109ms	remaining: 8.94s
12:	learn: 1285.9208633	total: 117ms	remaining: 8.89s
13:	learn: 1268.7546225	total: 126ms	remaining: 8.88s
14:	learn: 1252.8749351	total: 135ms	remaining: 8.86s
15:	learn: 1238.2544486	total: 144ms	remaining: 8.85s
16:	learn: 1224.1407072	total: 153ms	remaining: 8.83s
17:	learn: 1211.3477044	total: 162ms	remaining: 8.81s
18:	l

160:	learn: 1057.4133890	total: 1.43s	remaining: 7.46s
161:	learn: 1057.2456428	total: 1.44s	remaining: 7.46s
162:	learn: 1056.9424722	total: 1.45s	remaining: 7.45s
163:	learn: 1056.8116632	total: 1.46s	remaining: 7.44s
164:	learn: 1056.6020819	total: 1.47s	remaining: 7.43s
165:	learn: 1056.3992533	total: 1.48s	remaining: 7.42s
166:	learn: 1056.2509563	total: 1.49s	remaining: 7.41s
167:	learn: 1056.1405723	total: 1.5s	remaining: 7.42s
168:	learn: 1055.9856908	total: 1.51s	remaining: 7.42s
169:	learn: 1055.7871311	total: 1.52s	remaining: 7.4s
170:	learn: 1055.5955375	total: 1.52s	remaining: 7.4s
171:	learn: 1055.4592829	total: 1.53s	remaining: 7.38s
172:	learn: 1055.3206049	total: 1.54s	remaining: 7.38s
173:	learn: 1055.0930873	total: 1.55s	remaining: 7.36s
174:	learn: 1054.8666487	total: 1.56s	remaining: 7.35s
175:	learn: 1054.7399761	total: 1.57s	remaining: 7.34s
176:	learn: 1054.5980222	total: 1.58s	remaining: 7.33s
177:	learn: 1054.2977353	total: 1.58s	remaining: 7.32s
178:	learn: 1

323:	learn: 1030.9602659	total: 2.86s	remaining: 5.98s
324:	learn: 1030.8550771	total: 2.87s	remaining: 5.97s
325:	learn: 1030.6120944	total: 2.88s	remaining: 5.96s
326:	learn: 1030.4783176	total: 2.89s	remaining: 5.95s
327:	learn: 1030.3839404	total: 2.9s	remaining: 5.94s
328:	learn: 1030.2577588	total: 2.91s	remaining: 5.93s
329:	learn: 1030.0771555	total: 2.92s	remaining: 5.92s
330:	learn: 1029.9494752	total: 2.93s	remaining: 5.92s
331:	learn: 1029.7963460	total: 2.94s	remaining: 5.91s
332:	learn: 1029.5941424	total: 2.94s	remaining: 5.9s
333:	learn: 1029.3690946	total: 2.95s	remaining: 5.89s
334:	learn: 1029.1904291	total: 2.96s	remaining: 5.88s
335:	learn: 1028.9797097	total: 2.97s	remaining: 5.87s
336:	learn: 1028.7636698	total: 2.98s	remaining: 5.86s
337:	learn: 1028.6727102	total: 2.99s	remaining: 5.85s
338:	learn: 1028.5868849	total: 3s	remaining: 5.85s
339:	learn: 1028.4152130	total: 3.01s	remaining: 5.84s
340:	learn: 1028.2389052	total: 3.02s	remaining: 5.84s
341:	learn: 102

482:	learn: 1008.1194412	total: 4.3s	remaining: 4.6s
483:	learn: 1008.0017681	total: 4.31s	remaining: 4.59s
484:	learn: 1007.8197491	total: 4.32s	remaining: 4.58s
485:	learn: 1007.6990184	total: 4.32s	remaining: 4.57s
486:	learn: 1007.6057898	total: 4.33s	remaining: 4.56s
487:	learn: 1007.4484767	total: 4.34s	remaining: 4.55s
488:	learn: 1007.2758984	total: 4.35s	remaining: 4.55s
489:	learn: 1007.1902368	total: 4.36s	remaining: 4.54s
490:	learn: 1006.9687967	total: 4.37s	remaining: 4.53s
491:	learn: 1006.7741898	total: 4.38s	remaining: 4.52s
492:	learn: 1006.6382467	total: 4.38s	remaining: 4.51s
493:	learn: 1006.5611255	total: 4.39s	remaining: 4.5s
494:	learn: 1006.4275903	total: 4.4s	remaining: 4.49s
495:	learn: 1006.2254983	total: 4.41s	remaining: 4.48s
496:	learn: 1006.1181751	total: 4.42s	remaining: 4.47s
497:	learn: 1005.9794084	total: 4.43s	remaining: 4.46s
498:	learn: 1005.7638645	total: 4.44s	remaining: 4.46s
499:	learn: 1005.6754586	total: 4.45s	remaining: 4.45s
500:	learn: 10

645:	learn: 989.1926348	total: 5.72s	remaining: 3.14s
646:	learn: 989.0065273	total: 5.73s	remaining: 3.13s
647:	learn: 988.9051010	total: 5.74s	remaining: 3.12s
648:	learn: 988.8124196	total: 5.75s	remaining: 3.11s
649:	learn: 988.6767568	total: 5.76s	remaining: 3.1s
650:	learn: 988.5499730	total: 5.77s	remaining: 3.09s
651:	learn: 988.4890069	total: 5.78s	remaining: 3.08s
652:	learn: 988.3301183	total: 5.79s	remaining: 3.07s
653:	learn: 988.1716612	total: 5.79s	remaining: 3.06s
654:	learn: 988.1365878	total: 5.8s	remaining: 3.06s
655:	learn: 988.0341208	total: 5.81s	remaining: 3.05s
656:	learn: 987.9198216	total: 5.82s	remaining: 3.04s
657:	learn: 987.7619168	total: 5.83s	remaining: 3.03s
658:	learn: 987.6749745	total: 5.84s	remaining: 3.02s
659:	learn: 987.5542238	total: 5.84s	remaining: 3.01s
660:	learn: 987.4234449	total: 5.85s	remaining: 3s
661:	learn: 987.3416951	total: 5.86s	remaining: 2.99s
662:	learn: 987.2205336	total: 5.87s	remaining: 2.98s
663:	learn: 987.1215607	total: 5.

810:	learn: 971.9170739	total: 7.16s	remaining: 1.67s
811:	learn: 971.7626129	total: 7.17s	remaining: 1.66s
812:	learn: 971.6865163	total: 7.18s	remaining: 1.65s
813:	learn: 971.6268923	total: 7.19s	remaining: 1.64s
814:	learn: 971.5130001	total: 7.2s	remaining: 1.63s
815:	learn: 971.4101380	total: 7.21s	remaining: 1.63s
816:	learn: 971.3799169	total: 7.21s	remaining: 1.62s
817:	learn: 971.3027100	total: 7.22s	remaining: 1.61s
818:	learn: 971.1927418	total: 7.23s	remaining: 1.6s
819:	learn: 971.1117633	total: 7.24s	remaining: 1.59s
820:	learn: 970.9950595	total: 7.25s	remaining: 1.58s
821:	learn: 970.9390498	total: 7.26s	remaining: 1.57s
822:	learn: 970.7996441	total: 7.27s	remaining: 1.56s
823:	learn: 970.7537175	total: 7.27s	remaining: 1.55s
824:	learn: 970.6760799	total: 7.28s	remaining: 1.54s
825:	learn: 970.5510620	total: 7.29s	remaining: 1.54s
826:	learn: 970.4452645	total: 7.3s	remaining: 1.53s
827:	learn: 970.3082287	total: 7.31s	remaining: 1.52s
828:	learn: 970.1519290	total: 

972:	learn: 956.0293479	total: 8.59s	remaining: 238ms
973:	learn: 955.9048819	total: 8.6s	remaining: 230ms
974:	learn: 955.7605341	total: 8.61s	remaining: 221ms
975:	learn: 955.6927419	total: 8.62s	remaining: 212ms
976:	learn: 955.6195999	total: 8.63s	remaining: 203ms
977:	learn: 955.4818683	total: 8.63s	remaining: 194ms
978:	learn: 955.4116717	total: 8.64s	remaining: 185ms
979:	learn: 955.3695314	total: 8.65s	remaining: 177ms
980:	learn: 955.3217116	total: 8.66s	remaining: 168ms
981:	learn: 955.1927646	total: 8.67s	remaining: 159ms
982:	learn: 955.1306553	total: 8.68s	remaining: 150ms
983:	learn: 955.0830846	total: 8.69s	remaining: 141ms
984:	learn: 954.8417675	total: 8.7s	remaining: 132ms
985:	learn: 954.7652641	total: 8.71s	remaining: 124ms
986:	learn: 954.6576353	total: 8.71s	remaining: 115ms
987:	learn: 954.5399856	total: 8.72s	remaining: 106ms
988:	learn: 954.4746971	total: 8.73s	remaining: 97.1ms
989:	learn: 954.3775045	total: 8.74s	remaining: 88.3ms
990:	learn: 954.3228664	tota

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1667.75
1,FDW14,OUT017,1449.7
2,NCN55,OUT010,692.068
3,FDQ58,OUT017,2400.63
4,FDY38,OUT027,5804.46
...,...,...,...
5676,FDB58,OUT046,2214.39
5677,FDD47,OUT018,2532.26
5678,NCO17,OUT045,1920.85
5679,FDJ26,OUT017,3560.44


## Hyperparameter tuning using RandomizedSearchCV 

In [104]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV

In [105]:
#figuring out the ideal value of CV
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
#Defining search space
param_grid={"min_samples_leaf" : [1, 5, 10], "min_samples_split" : [2, 4, 10,12], "n_estimators": [50, 70,120,150]}
rs=RandomizedSearchCV(reg_rf, param_grid, n_iter=500, scoring='neg_root_mean_squared_error', cv=cv, verbose=2)
#execute search
rs=rs.fit(X_train, y_train)
#summarize result
print("Best score: %s" % rs.best_score_)
print("Best Hyperparameters: %s" % rs.best_params_)

Fitting 30 folds for each of 48 candidates, totalling 1440 fits
[CV] n_estimators=50, min_samples_split=2, min_samples_leaf=1 ........


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=50, min_samples_split=2, min_samples_leaf=1, total=  16.5s
[CV] n_estimators=50, min_samples_split=2, min_samples_leaf=1 ........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.4s remaining:    0.0s


[CV]  n_estimators=50, min_samples_split=2, min_samples_leaf=1, total=  15.8s
[CV] n_estimators=50, min_samples_split=2, min_samples_leaf=1 ........
[CV]  n_estimators=50, min_samples_split=2, min_samples_leaf=1, total=  15.8s
[CV] n_estimators=50, min_samples_split=2, min_samples_leaf=1 ........
[CV]  n_estimators=50, min_samples_split=2, min_samples_leaf=1, total=  15.9s
[CV] n_estimators=50, min_samples_split=2, min_samples_leaf=1 ........
[CV]  n_estimators=50, min_samples_split=2, min_samples_leaf=1, total=  16.0s
[CV] n_estimators=50, min_samples_split=2, min_samples_leaf=1 ........
[CV]  n_estimators=50, min_samples_split=2, min_samples_leaf=1, total=  15.9s
[CV] n_estimators=50, min_samples_split=2, min_samples_leaf=1 ........
[CV]  n_estimators=50, min_samples_split=2, min_samples_leaf=1, total=  16.1s
[CV] n_estimators=50, min_samples_split=2, min_samples_leaf=1 ........
[CV]  n_estimators=50, min_samples_split=2, min_samples_leaf=1, total=  15.9s
[CV] n_estimators=50, min_sa

...

[CV]  n_estimators=50, min_samples_split=12, min_samples_leaf=10, total=   8.7s
[CV] n_estimators=50, min_samples_split=12, min_samples_leaf=10 ......
[CV]  n_estimators=50, min_samples_split=12, min_samples_leaf=10, total=   8.6s
[CV] n_estimators=50, min_samples_split=12, min_samples_leaf=10 ......
[CV]  n_estimators=50, min_samples_split=12, min_samples_leaf=10, total=   8.6s
[CV] n_estimators=50, min_samples_split=12, min_samples_leaf=10 ......
[CV]  n_estimators=50, min_samples_split=12, min_samples_leaf=10, total=   8.7s
[CV] n_estimators=50, min_samples_split=12, min_samples_leaf=10 ......
[CV]  n_estimators=50, min_samples_split=12, min_samples_leaf=10, total=   8.7s
[CV] n_estimators=50, min_samples_split=12, min_samples_leaf=10 ......
[CV]  n_estimators=50, min_samples_split=12, min_samples_leaf=10, total=   8.7s
[CV] n_estimators=50, min_samples_split=12, min_samples_leaf=10 ......
[CV]  n_estimators=50, min_samples_split=12, min_samples_leaf=10, total=   8.8s
[CV] n_estimat

[CV]  n_estimators=70, min_samples_split=12, min_samples_leaf=10, total=  12.2s
[CV] n_estimators=70, min_samples_split=12, min_samples_leaf=10 ......
[CV]  n_estimators=70, min_samples_split=12, min_samples_leaf=10, total=  12.6s
[CV] n_estimators=70, min_samples_split=12, min_samples_leaf=10 ......
[CV]  n_estimators=70, min_samples_split=12, min_samples_leaf=10, total=  12.5s
[CV] n_estimators=70, min_samples_split=12, min_samples_leaf=10 ......
[CV]  n_estimators=70, min_samples_split=12, min_samples_leaf=10, total=  12.1s
[CV] n_estimators=70, min_samples_split=12, min_samples_leaf=10 ......
[CV]  n_estimators=70, min_samples_split=12, min_samples_leaf=10, total=  12.1s
[CV] n_estimators=120, min_samples_split=12, min_samples_leaf=10 .....
[CV]  n_estimators=120, min_samples_split=12, min_samples_leaf=10, total=  21.4s
[CV] n_estimators=120, min_samples_split=12, min_samples_leaf=10 .....
[CV]  n_estimators=120, min_samples_split=12, min_samples_leaf=10, total=  20.7s
[CV] n_estim

[CV]  n_estimators=150, min_samples_split=12, min_samples_leaf=10, total=  34.0s
[CV] n_estimators=150, min_samples_split=12, min_samples_leaf=10 .....
[CV]  n_estimators=150, min_samples_split=12, min_samples_leaf=10, total=  34.2s
[CV] n_estimators=150, min_samples_split=12, min_samples_leaf=10 .....
[CV]  n_estimators=150, min_samples_split=12, min_samples_leaf=10, total=  34.4s
[CV] n_estimators=150, min_samples_split=12, min_samples_leaf=10 .....
[CV]  n_estimators=150, min_samples_split=12, min_samples_leaf=10, total=  34.6s
[CV] n_estimators=150, min_samples_split=12, min_samples_leaf=10 .....
[CV]  n_estimators=150, min_samples_split=12, min_samples_leaf=10, total=  34.5s
[CV] n_estimators=150, min_samples_split=12, min_samples_leaf=10 .....
[CV]  n_estimators=150, min_samples_split=12, min_samples_leaf=10, total=  34.9s
[CV] n_estimators=150, min_samples_split=12, min_samples_leaf=10 .....
[CV]  n_estimators=150, min_samples_split=12, min_samples_leaf=10, total=  34.1s
[CV] n_

[Parallel(n_jobs=1)]: Done 1440 out of 1440 | elapsed: 3015.7min finished


Best score: -1110.902446832914
Best Hyperparameters: {'n_estimators': 120, 'min_samples_split': 2, 'min_samples_leaf': 10}


## Hyperparameter tuning for XGboost model

In [ ]:
#figuring out the ideal value of CV
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
#Defining search space
param_grid={"learning_rate" : [0.01,0.1], "max_depth" : [3,5,7,10], "min_child_weight": [1,3,5], "subsample": [0.5,0.7], "n_estimators": [20,50, 70, 100]}
xgb=RandomizedSearchCV(reg_xgb, param_grid, n_iter=500, scoring='neg_root_mean_squared_error', cv=cv, verbose=2)
#execute search
xgb=xgb.fit(X_train, y_train)
#summarize result
print("Best score: %s" % xgb.best_score_)
print("Best Hyperparameters: %s" % xgb.best_params_)

Fitting 30 folds for each of 192 candidates, totalling 5760 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] subsample=0.5, n_estimators=20, min_child_weight=1, max_depth=3, learning_rate=0.01 
[10:59:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.5, n_estimators=20, min_child_weight=1, max_depth=3, learning_rate=0.01, total=   4.6s
[CV] subsample=0.5, n_estimators=20, min_child_weight=1, max_depth=3, learning_rate=0.01 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.5s remaining:    0.0s


[10:59:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.5, n_estimators=20, min_child_weight=1, max_depth=3, learning_rate=0.01, total=   2.2s
[CV] subsample=0.5, n_estimators=20, min_child_weight=1, max_depth=3, learning_rate=0.01 
[10:59:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.5, n_estimators=20, min_child_weight=1, max_depth=3, learning_rate=0.01, total=   1.8s
[CV] subsample=0.5, n_estimators=20, min_child_weight=1, max_depth=3, learning_rate=0.01 
[10:59:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.5, n_estimators=20, min_child_weight=1, max_depth=3, learning

...

[11:56:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.5, n_estimators=50, min_child_weight=1, max_depth=10, learning_rate=0.1, total=  10.1s
[CV] subsample=0.5, n_estimators=50, min_child_weight=1, max_depth=10, learning_rate=0.1 
[11:56:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.5, n_estimators=50, min_child_weight=1, max_depth=10, learning_rate=0.1, total=   9.7s
[CV] subsample=0.5, n_estimators=50, min_child_weight=1, max_depth=10, learning_rate=0.1 
[11:56:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.5, n_estimators=50, min_child_weight=1, max_depth=10, learnin

[CV]  subsample=0.7, n_estimators=50, min_child_weight=1, max_depth=10, learning_rate=0.1, total=   9.8s
[CV] subsample=0.7, n_estimators=50, min_child_weight=1, max_depth=10, learning_rate=0.1 
[11:59:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.7, n_estimators=50, min_child_weight=1, max_depth=10, learning_rate=0.1, total=   9.6s
[CV] subsample=0.7, n_estimators=50, min_child_weight=1, max_depth=10, learning_rate=0.1 
[12:00:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.7, n_estimators=50, min_child_weight=1, max_depth=10, learning_rate=0.1, total=   9.5s
[CV] subsample=0.7, n_estimators=50, min_child_weight=1, max_depth=10, learning_rate=0.1 
[12:00:11] WARNING: C:/Users/Administrator/workspace/xgboost-

[12:03:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.5, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1, total=  14.8s
[CV] subsample=0.5, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1 
[12:04:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.5, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1, total=  14.4s
[CV] subsample=0.5, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1 
[12:04:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.5, n_estimators=70, min_child_weight=1, max_depth=10, learnin

[CV]  subsample=0.7, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1, total=  15.8s
[CV] subsample=0.7, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1 
[12:09:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.7, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1, total=  15.5s
[CV] subsample=0.7, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1 
[12:09:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.7, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1, total=  15.3s
[CV] subsample=0.7, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1 
[12:09:56] WARNING: C:/Users/Administrator/workspace/xgboost-

[12:14:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.7, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1, total=  13.3s
[CV] subsample=0.7, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1 
[12:14:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.7, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1, total=  13.3s
[CV] subsample=0.7, n_estimators=70, min_child_weight=1, max_depth=10, learning_rate=0.1 
[12:15:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  subsample=0.7, n_estimators=70, min_child_weight=1, max_depth=10, learnin